In [33]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ast


df=pd.read_csv('tmdb_5000_movies.csv')

def extract_genre_names(genres_str):
    try:
        genres_list = ast.literal_eval(genres_str)
        genre_names = [genre['name'] for genre in genres_list]
        return ','.join(genre_names)
    except:
        return ''

df['genres_clean'] = df['genres'].apply(extract_genre_names)
df['genres_clean'] = df['genres_clean'].str.lower()
unique=df['genres'].unique()
unique_lang=df['original_language'].unique()
df.head()
label=LabelEncoder()
df['encoded genres']=label.fit_transform(df['genres_clean'])
label2=LabelEncoder()
df['encoded title']=label2.fit_transform(df['title'])
x=df[['encoded genres','popularity']]
y=df['encoded title']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
model=KNeighborsClassifier(n_neighbors=3)
model.fit(x_train,y_train)
all_genre = [
    'action,adventure,fantasy,science fiction',
    'adventure,fantasy,action',
    'adventure,crime',
    'thriller,horror,comedy',
    'foreign,thriller',
    'comedy,drama,romance,tv movie'
]
print('types of genre:',all_genre)
users_genre=input('enter genre :')
print('popularity from (0-875)')
users_pop=float(input('enter popuarity: '))
users_genre_enc=label.transform([users_genre])[0]
feature=pd.DataFrame([[users_genre_enc,users_pop]],columns=['encoded genres', 'popularity'])
title_enc=model.predict(feature)[0]
title=label2.inverse_transform([title_enc])[0]
print(f'predicted movie title: {title}')

#print(unique)


types of genre: ['action,adventure,fantasy,science fiction', 'adventure,fantasy,action', 'adventure,crime', 'thriller,horror,comedy', 'foreign,thriller', 'comedy,drama,romance,tv movie']
enter genre :comedy
popularity from (0-875)
enter popuarity: 150
predicted movie title: Brave


In [12]:
top_popularities = df['popularity'].nlargest(5)
print(top_popularities)


546    875.581305
95     724.247784
788    514.569956
94     481.098624
127    434.278564
Name: popularity, dtype: float64


In [13]:
unique_genres=df['genres_clean'].unique()
print(unique_genres)

['action,adventure,fantasy,science fiction' 'adventure,fantasy,action'
 'action,adventure,crime' ... 'thriller,horror,comedy' 'foreign,thriller'
 'comedy,drama,romance,tv movie']


In [35]:
import numpy as np

# Create DataFrame for prediction
feature = pd.DataFrame([[users_genre_enc, users_pop]],
                       columns=['encoded genres', 'popularity'])

# Predict label (encoded title)
predicted_label = model.predict(feature)[0]

# Find the position (index) of this label in model.classes_
predicted_class_pos = np.where(model.classes_ == predicted_label)[0][0]

# Get probability for that position
predicted_class_prob = model.predict_proba(feature)[0][predicted_class_pos] * 100

# Decode movie title
predicted_title = label2.inverse_transform([predicted_label])[0]

print(f"Predicted movie: {predicted_title}")
print(f"Confidence: {predicted_class_prob:.2f}%")


Predicted movie: Brave
Confidence: 33.33%
